In [1]:
# set environment variables if not defined globally
import os
os.environ["DEEPTHOUGHT_DATA_PATH"]="/home/stober/deepthought/datasets"
os.environ["DEEPTHOUGHT_OUTPUT_PATH"]="/home/stober/deepthought/output"
os.environ["DEEPTHOUGHT_ROOT"]="/home/stober/deepthought"

os.environ["THEANO_FLAGS"] = "floatX=float32,device=gpu0"  # set to cpu if you don't have a suitable GPU with CUDA

import deepthought

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5005)


In [2]:
from deepthought.datasets.openmiir.constants import STIMULUS_IDS

# subjects = ['P01','P04','P05','P06','P07','P09','P11','P12','P13','P14']  
# subjects = ['P01','P04','P06','P07','P09','P11','P12','P13','P14']  # reject P05 (coughing artifacts)
subjects = ['P09','P11','P12','P13','P14']  # reject P01..P07 (slightly different stimulus tempo)

stimuli = STIMULUS_IDS
cond = 1
extra = 0  # add extra time before start of epoch?
include_cue = True

eeg_export_basepath = 'data/raw'
beats_export_basepath = 'data/beats'
sce_export_basepath = 'data/sce-filtered'

In [3]:
## modified auxiliary function with addional option to add some time before and after trials
def get_trial_epochs(raw, trial_events, stim_id, condition, extra=0,
                     subject=None, stimuli_version=None, meta=None,
                     include_cue=False, picks=None, debug=False):
    
    from deepthought.datasets.openmiir.preprocessing.events import filter_trial_events

    assert subject is None or stimuli_version is None or meta is None

    if meta is None:
        if stimuli_version is None:
            if subject is None:
                raise RuntimeError('Either meta, stimuli_version or subject has to be specified.')
            else:
                stimuli_version = get_stimuli_version(subject)
        meta = load_stimuli_metadata(version=stimuli_version)

    events = filter_trial_events(trial_events, [stim_id], [condition])
    if debug:
        print('selected events: {}'.format(len(events)))
        print(events)

    start = -extra
    if condition in [1,2]: # cued
        if include_cue:
            stop = meta[stim_id]['length_with_cue']
        else:
            # NOTE: start > 0 does not work; need to shift event time
            offset = int(np.floor(meta[stim_id]['length_of_cue'] * raw.info['sfreq']))
            events[:,0] += offset
            stop = meta[stim_id]['length_without_cue']
    else:
        stop = meta[stim_id]['length_without_cue']
        
    stop += extra

    if debug:
        print('start: {}  stop: {}'.format(start, stop))
        print(events)

    if picks is None:
        # default: all EEG channels including bad/interpolated
        picks = mne.pick_types(raw.info, meg=False, eeg=True, eog=False, stim=False, exclude=[])

    epochs = mne.Epochs(raw, events, None,
                              tmin=start, tmax=stop, preload=True,
                              proj=False, picks=picks, verbose=False)

    if debug:
        print(epochs)

    return epochs

In [4]:
## reset cache
cache = dict()

In [5]:
def load(subject):
    from deepthought.datasets.openmiir.preprocessing import load_and_preprocess_raw
    return load_and_preprocess_raw(subject, 
                                  l_freq=None, h_freq=None, 
                                  onsets='audio',
                                  interpolate_bad_channels=True,
                                  #reference_mastoids=True, # important!!! cannot combine different references
                                  reference_mastoids=False, # important!!! cannot combine different references
                                  sfreq=512,
                                  ica_cleaning=True,
#                                   ica_cleaning=False,
#                                   l_freq2=0.5, h_freq2=8,
                                  n_jobs=4,
                                  verbose=True)

In [ ]:
## optional: preload data into cache
for subject in subjects:
    break
    
    if subject in cache:
        continue
    raw, trial_events = load(subject)
    cache[subject] = raw, trial_events

In [7]:
def build_encoder_model():    
    import theano
    import numpy as np
    from blocks.bricks import Tanh
    from blocks.bricks.conv import Convolutional
    from blocks.initialization import Constant
    from blocks.model import Model
    
    input_ = theano.tensor.tensor4(dtype=theano.config.floatX)
    
    conv = Convolutional(name='conv',
                         filter_size=(1, 1), 
                         num_filters=1, # out
                         num_channels=64, # in
                         image_size=(None, 1), 
                         weights_init=Constant(0), # irrelevant
                         use_bias=False)
    conv.initialize()
    
    weights = np.load('sce-weights.npy')
    conv.W.set_value(weights)

    tanh = Tanh()    
    output = tanh.apply(conv.apply(input_))
    
    encoder = Model(output)    
    encoder_fn = theano.function(inputs=[input_], outputs=output, allow_input_downcast=True)
    
    return encoder, encoder_fn

encoder, encoder_fn = build_encoder_model()

In [8]:
import mne
from deepthought.datasets.openmiir.events import get_event_id
# from deepthought.datasets.openmiir.epochs import get_trial_epochs
from deepthought.datasets.openmiir.metadata import load_beat_times, get_stimuli_version, load_stimuli_metadata
import deepthought.util.fs_util as fs_util
from scipy import io
import os
import numpy as np

for subject in subjects:
    if subject in cache:
        raw, trial_events = cache[subject]
    else:
        raw, trial_events = load(subject)

    for stim in stimuli:
        ## EEG data
        event_id = get_event_id(stim, cond)            
        epochs = get_trial_epochs(raw, trial_events, stim, cond, 
                                  extra=extra, subject=subject, include_cue=include_cue, debug=False)
        print epochs, epochs._data.shape
        filename = os.path.join(eeg_export_basepath, 'cond{}/{:0>2}_{}.mat'.format(cond, stim, subject))
        fs_util.ensure_parent_dir_exists(filename)
        io.savemat(filename, dict(data=epochs._data), oned_as='row')
        print filename
        
        ## beat times
        version = get_stimuli_version(subject)
        if include_cue:
            cue_beats = load_beat_times(stim, cue=True, version=version)
            offset = load_stimuli_metadata(version=version)[stim]['length_of_cue']
        else:
            cue_beats = []
            offset = 0
            
        noncue_beats = load_beat_times(stim, cue=False, version=version) + offset
        beats = np.concatenate([cue_beats, noncue_beats])
    
        filename = os.path.join(beats_export_basepath, 'cond{}/{:0>2}_{}.txt'.format(cond, stim, subject))
        fs_util.ensure_parent_dir_exists(filename)
        with open(filename, 'w') as f:
            for beat in beats:
                f.write('{}\n'.format(beat))
        print filename
        
        # SCE-encoded trials
        data_bc01 = np.reshape(epochs._data, epochs._data.shape + (1,))
        # print data_bc01.shape        
        encoded = encoder_fn(data_bc01).squeeze()
        
        filename = os.path.join(sce_export_basepath, 'cond{}/{:0>2}_{}.mat'.format(cond, stim, subject))
        fs_util.ensure_parent_dir_exists(filename)
        io.savemat(filename, dict(data=encoded), oned_as='row')
        print filename, subject, stim, encoded.shape

Opening raw data file /home/stober/deepthought/datasets/OpenMIIR/eeg/mne/P09-raw.fif...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 66)  idle
    Range : 0 ... 2440555 =      0.000 ...  4766.709 secs
Ready.
Reading 0 ... 2440555  =      0.000 ...  4766.709 secs...
540 events found
Events id: [  11   12   13   14   21   22   23   24   31   32   33   34   41   42   43
   44  111  112  113  114  121  122  123  124  131  132  133  134  141  142
  143  144  211  212  213  214  221  222  223  224  231  232  233  234  241
  242  243  244 1000 1111 2000 2001]
360 events found
Events id: [  11   12   13   14   21   22   23   24   31   32   33   34   41   42   43
   44  111  112  113  114  121  122  123  124  131  132  133  134  141  142
  143  144  211  212  213  214  221  222  223  224  231  232  233  234  241
  242  243  244 1111 2000 2001]
Reading /home/stober/deepthought/datasets/OpenMIIR/eeg/preprocessing/ica/P09-100p_64c-ica.fif ...
Isotrak